# lets implement polynomial linear regression from scratch


## import section

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from itertools import combinations_with_replacement

In [ ]:
reg_dataset = datasets.make_regression(n_samples=1000, n_features=100, n_informative=10, n_targets=1, shuffle=True, random_state=42)
X = reg_dataset[0]
y = reg_dataset[1]

# lets write code for polynomial features

In [ ]:
class PolynomialFeatures:

	def __init__(self, degree = 2):

		"""
			Args: degree = 2(default)
		"""

		self.degree = degree
		self.num_input_feats = int 
		self.num_output_feats = int

	def fit_transform(self, X):

		"""
			Generates new feature matrix consisiting of all combinations of
			the features with degree less than or equal to specified degree(or 2 by default) 
			Ex. input sample = [a, b]
			    output sample(with degree 3) = [1, a, b, a^2, ab, b^2, a^3, a^2b, ab^2, b^3]
			Args: 
				X (np.array) - matrix with features to be transformed
			Returns:
				X_new (np.array) - matrix with transformed features
		"""
		X = pd.DataFrame(X)
		sample_size, self.num_feats = np.shape(X)

		combs_obj = [combinations_with_replacement(range(np.shape(X)[1]), i) for i in range(0, self.degree + 1)]
		#print(combs_obj)
		combinations = [item  for combination in combs_obj for item in combination]
		#print(combinations)
		self.num_output_feats = len(combinations)

		X_new = np.empty((sample_size, self.num_output_feats))

		#Note: np.prod([]) = 1.0 (The product of an empty array is the neutral element 1.0)
		for i, index_combs in enumerate(combinations):
			index_combs = list(index_combs)
			X_new[:, i] = np.prod(X.iloc[:, index_combs], axis=1)

		return pd.DataFrame(X_new)

In [ ]:
poly_obj = PolynomialFeatures()
X_new = poly_obj.fit_transform(X)
X_new = np.array(X_new.values)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_new,y,train_size = .80,shuffle= True, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(800, 5151)
(200, 5151)
(800,)
(200,)


In [ ]:
standardscalar = StandardScaler()
standardscalar.fit(X_train)
standardscalar.transform(X_train)
standardscalar.transform(X_test)


array([[ 0.        ,  0.75070067, -0.3481925 , ..., -0.56397695,
         0.24040667, -0.44870833],
       [ 0.        ,  0.84896602,  0.31452426, ...,  1.83670648,
         0.71461063, -0.60262078],
       [ 0.        ,  0.66993726,  1.23443248, ..., -0.03534746,
         1.17012182,  0.38762077],
       ...,
       [ 0.        , -0.62975098,  0.57138191, ...,  0.51209028,
        -0.28101969, -0.6773484 ],
       [ 0.        , -0.84372868, -1.52121785, ...,  1.27788636,
        -0.1527934 , -0.70372273],
       [ 0.        ,  0.29565289,  0.92703245, ..., -0.32272852,
        -0.87092117,  0.3415663 ]])

# lets write code for linear regression

In [ ]:
class LinearRegression:

	""" Linear Regression with multiple features """

	def __init__(self, learning_rate = 1e-3, max_iter = 1000):

		self.num_feats = int
		self.train_size = int
		self.weights = np.array 
		self.y_train = np.array 
		self.input_matrix = np.array

		self.learning_rate = learning_rate   #Learning rate for gradient descent
		self.max_iter = max_iter 	#Number of iterations to run gradient descent
		self.cost_threshold = 0.1 * learning_rate  #stopping criterion for gradien descent

	def fit(self, X, y):

		"""
			Adjust weights to training data
		"""

		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]
		self.input_matrix = np.append(X, np.ones(self.train_size).reshape(-1, 1), axis = 1)   #Add Column with Ones for intercept term 
		self.y_train = y
		self.weights = np.zeros(self.num_feats + 1) #Extra +1 for the intercept


		#optimize weights
		prev_cost = float("inf")
		for i in range(self.max_iter):
			cost = self._update_weights()

			if i%100 ==0 or i == self.max_iter:
				print("Cost after {} iterations is: {}".format(i, cost))
			if abs(prev_cost -cost) < self.cost_threshold*prev_cost:
				print("Cost after {} iterations is: {}".format(i, cost))
				break
			prev_cost = cost

	def _update_weights(self):

		"""
			Cost Function:
				l(w) = (1/n) * (((y - wX)^2) 
			Gradient:
				delta_w = dl/dw = (2/n)*( ((y - wX)*(-X))
							
							 (or)
				delta_w = dl/dw = (2/n)*( ((wX - y)*(X)) 
			Gradient Descent:
				w = w - (learning_rate * delta_w)
		"""

		y_pred = (self.weights * self.input_matrix).sum(axis = 1)  # y_pred = wX

		cost = (1/self.train_size) * (((self.y_train - y_pred) ** 2).sum(axis = 0))  

		err = (y_pred - self.y_train).reshape(-1, 1)  # err = wX - y

		delta_w = (2/self.train_size) * ((err * self.input_matrix).sum(axis = 0)) #delta_w = (2/n)*( (wX - y)*(X)) 

		self.weights = self.weights - (self.learning_rate * delta_w) 

		return cost


	def predict(self, X):

		""" Make predictions on given X using trained model """

		size = X.shape[0]
		X = np.append(X, np.ones(size).reshape(-1, 1), axis = 1)

		y_pred = (self.weights * X).sum(axis = 1)

		return y_pred 

In [ ]:
lin_reg = LinearRegression(learning_rate = 1e-3, max_iter = 1000)
lin_reg.fit(X_train, y_train)

Cost after 0 iterations is: 24752.939027679462
Cost after 100 iterations is: 2070.9001491751433
Cost after 200 iterations is: 400.97332286608855
Cost after 300 iterations is: 107.89324950643638
Cost after 400 iterations is: 33.876310292847606
Cost after 500 iterations is: 11.609961717727183
Cost after 600 iterations is: 4.209826166854914
Cost after 700 iterations is: 1.5882033748620836
Cost after 800 iterations is: 0.6171843468153716
Cost after 900 iterations is: 0.24547387327089895


In [ ]:
print('Linear Regression Model Coefficients (W): {}'.format(lin_reg.weights[:-1]))
print('Linear Regression Model Intercept (b): {}'.format(lin_reg.weights[-1]))

Linear Regression Model Coefficients (W): [-0.23469747  8.51077838  0.37556372 ...  0.62167612  0.23031805
  2.68156497]
Linear Regression Model Intercept (b): -0.23469746694905055


# lets see the MAPE of the model

In [ ]:
#Evaluating Model through MAPE
print("\nMean Absolute Percentage Error(for train data): {}".format(mean_absolute_percentage_error(y_train, lin_reg.predict(X_train))))
print("Mean Absolute Percentage Error(for test data): {}".format(mean_absolute_percentage_error(y_test, lin_reg.predict(X_test))))


Mean Absolute Percentage Error(for train data): 0.0049099504741054525
Mean Absolute Percentage Error(for test data): 1.6776840649041842
